In [ ]:
# カテゴリカルフィーチャのための CatBoost 符号化。

# これはleave-one-one-out符号化に非常に似ていますが、
# 値を "オンザフライ "で計算します。そのため、学習中に値は自然に変化し、
# ランダムなノイズを追加する必要はありません。

# 注意してほしいのは、
# 学習データはランダムに順列化されている必要があるということです。
# 例えば、以下のようになります。
# ランダム順列 
# ラpermutation perm = np.random.permutation(len(X)) X = X.iloc[perm].reset_index(drop=True) y = y.iloc[perm].reset_index(drop=True)

#※Leave One Out：
#カテゴリ特徴のコーディングを 1 つ除外します。
#これはターゲット・エンコーディングに非常に似ていますが、
#レベルの平均ターゲットを計算する際に現在の行のターゲットを除外して、外れ値の影響を軽減します。

# ・Count Encoding 登場回数のカウント
# ・Target Encoding SUM(y)/登場回数
# 　※Leakになるのでtestデータにはfit厳禁
# ・Leave-one-out Encoding その前の行までのSUM(y)/登場回数　
# 　※要シャッフル。Leakになるのでtestデータにはfit厳禁
# ・CatBoost Encoding　その前の行までのSUM(y)/登場回数＋補正
# 　※要シャッフル。Leakになるのでtestデータにはfit厳禁
# ・Feature embedding with SVD 

In [3]:
import numpy as np
import pandas as pd              # version 0.23.4
import category_encoders as ce   # version 1.2.8

pd.options.display.float_format = '{:.2f}'.format # to make legible

# some new data frame for the contrast encoders
df3 = pd.DataFrame(
    {'color':[3,2,0,1,1,1,2,2,3,3,4,5,6,7,8,3], 
     'outcome':[1,0,0,1,2,1,1,0,0,1,0,2,2,1,1,1]})

# set up X and y
X3 = df3.drop('outcome', axis = 1)
y3 = df3.drop('color', axis = 1)

df3

,color,outcome
0,3,1
1,2,0
2,0,0
3,1,1
4,1,2
5,1,1
6,2,1
7,2,0
8,3,0
9,3,1


In [27]:
# Target with default parameters
ce_target = ce.CatBoostEncoder(cols = ['color'])

ce_target.fit(X3, y3['outcome'])
# Must pass the series for y in v1.2.8

df3["ce"] = ce_target.transform(X3, y3['outcome'])
df3.sort_values("color")
# どのcolorもはじめは0.88だな、その前の行がないから？

,color,outcome,ce
2,0,0,0.88
3,1,1,0.88
4,1,2,0.94
5,1,1,1.29
1,2,0,0.88
6,2,1,0.44
7,2,0,0.62
0,3,1,0.88
8,3,0,0.94
9,3,1,0.62


In [31]:
i = 0
print( df3[df3["color"]==i]["outcome"].sum() / len(df3[df3["color"]==i]) )
print( df3[df3["color"]==i][["outcome","ce"]] )

0.0
   outcome   ce
2        0 0.88


In [52]:
i = 1
print( df3[df3["color"]==i]["outcome"].sum() / len(df3[df3["color"]==i]) )
print( df3[df3["color"]==i][["outcome","ce"]] )

print("1行目までのSUM/2行+α", 1/2 + 0.88/2 )
print("2行目までのSUM/3行+α", 4/3)

1.3333333333333333
   outcome   ce
3        1 0.88
4        2 0.94
5        1 1.29
1行目までのSUM/2行+α 0.94
2行目までのSUM/3行+α 1.3333333333333333


In [50]:
i = 2
print( df3[df3["color"]==i]["outcome"].sum() / len(df3[df3["color"]==i]) )
print( df3[df3["color"]==i][["outcome","ce"]] )

print("1行目までのSUM/2行+α", 0/2 + 0.88/2 )
print("2行目までのSUM/3行+α", 1/3 + 0.88/3)　# 自分が0だった場合も+アルファするのかな？？

0.3333333333333333
   outcome   ce
1        0 0.88
6        1 0.44
7        0 0.62
1行目までのSUM/2行+α 0.44
2行目までのSUM/3行 0.6266666666666667


In [48]:
i = 3
print( df3[df3["color"]==i]["outcome"].sum() / len(df3[df3["color"]==i]) )
print( df3[df3["color"]==i][["outcome","ce"]] )

print("1行目までのSUM/2行+α", 1/2 + 0.88/2 )
print("2行目までのSUM/3行", 2/3)
print("3行目までのSUM/3行", 3/4)

0.75
    outcome   ce
0         1 0.88
8         0 0.94
9         1 0.62
15        1 0.72
1行目までのSUM/2行+α 0.94
2行目までのSUM/3行 0.6666666666666666
3行目までのSUM/3行 0.75
